Was used to create daily weather2, a replacement for daily weather (now A6. Weather) for backtesting. I don't know if it's going to be necessary going forward

In [1]:
%reset -f
import random
import numpy as np
import pandas as pd
import os
import xlrd
import import_ipynb
import math
import glob
import re
from interruptingcow import timeout
# import warnings
# warnings.simplefilter(action="ignore")
import datetime
from datetime import date
import time 
import pickle
from Classes import *
from simulation_functions_py import *
from functools import partial
from joblib import Parallel, delayed

importing Jupyter notebook from Classes.ipynb
Code was last run on: 2023-03-05


In [2]:
baseball_path = r"C:\Users\james\Documents\MLB\Data"

In [3]:
# This reads in a map of team name, codes, and the number Fangraphs uses in their URLs
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

# We just need teams right now
team_map = team_map[['VENUE_ID', 'BBREFTEAM', 'FANGRAPHSTEAM', 'CF']]

In [4]:
# This reverses winds so that they're named for where they're going, not where they're from. This is so vectors make more sense logically.
def wind_reverser(direction):
    direction = direction.replace("N", "s")
    direction = direction.replace("S", "n")
    direction = direction.replace("E", "w")
    direction = direction.replace("W", "e")
    
    return direction.upper()

In [5]:
# This calculates number of degrees for each direction
def find_degree(direction):
    if direction == "N":
        degree = 0
    elif direction == "NNE":
        degree = 1
    elif direction == "NE":
        degree = 2
    elif direction == "ENE":
        degree = 3
    elif direction == "E":
        degree = 4
    elif direction == "ESE":
        degree = 5
    elif direction == "SE":
        degree = 6
    elif direction == "SSE":
        degree = 7
    elif direction == "S":
        degree = 8
    elif direction == "SSW":
        degree = 9
    elif direction == "SW":
        degree = 10
    elif direction == "WSW":
        degree = 11
    elif direction == "W":
        degree = 12
    elif direction == "WNW":
        degree = 13
    elif direction == "NW":
        degree = 14
    elif direction == "NNW":
        degree = 15
        
    degree = degree * 22.5 

    return degree

In [6]:
# This calculates the x and y vectors given the park's orientation and the wind's direction
def calculate_vectors(row):
    # Determines degree of centerfield
    park_angle = find_degree(row['CF'])
    # Determine degree of wind
    row['Direction'] = wind_reverser(row['Direction'])
    wind_angle = find_degree(row['Direction']) 
    
    # Determine angle between them
    angle = wind_angle - park_angle 

    # Calculate vectors
    x_vect = round(math.sin(math.radians(angle)), 5) * row['Speed']
    y_vect = round(math.cos(math.radians(angle)), 5) * row['Speed']

    return x_vect, y_vect

In [7]:
# # 
# date = "20220914"
# matchup_path = "Matchups" + date
# filename = "NYY_BOS 09142022 0710PM ET.xlsx"
# df = pd.read_excel(os.path.join(baseball_path, "New_Matchups", matchup_path, filename), sheet_name='HomeBatters')
# df = df.merge(team_map, on='BBREFTEAM', how='left')

# df = df[['venue_id', 'game_date', 'weather', 'wind', 'BBREFTEAM', 'CF']].iloc[[0]]
# df
# # x_vect, y_vect = calculate_vectors(df)
# # df[['x_vect', 'y_vect']] = df.apply(calculate_vectors, axis=1, result_type='expand')

# # df


In [8]:
def create_park_dummies():
    all_parks = pd.read_csv(os.path.join(baseball_path, "API", "Inputs", "All Parks.csv"))
    venue_dummies = pd.get_dummies(all_parks['venue_id'], prefix='venue')
    all_parks = all_parks[['venue_id']]
    all_parks = pd.concat([all_parks, venue_dummies], axis=1)
    return all_parks

all_parks = create_park_dummies()

In [9]:
def create_weather(matchup_path, file):
    df = pd.read_excel(os.path.join(baseball_path, "New_Matchups", matchup_path, file), sheet_name='HomeBatters')
    df = df.merge(team_map, on='BBREFTEAM', how='left')

    df = df[['venue_id', 'game_date', 'weather', 'wind', 'BBREFTEAM', 'CF']].iloc[[0]]
    
    df['TEMP_PARK_CT'] = df['weather'].str.extract('(\d+)').astype(int)
    df['Speed'] = df['wind'].str.extract('(\d+)').astype(int)
    
    
    df['CF_angle']  = df.apply(lambda x: find_degree(x['CF']), axis=1, result_type='expand')
    
    df['wind_angle'] = np.where(df['wind'].str.contains("Out To CF"), 0, 0)    
    df['wind_angle'] = np.where(df['wind'].str.contains("Out To RF"), 45, df['wind_angle'])
    df['wind_angle'] = np.where(df['wind'].str.contains("L To R"), 90, df['wind_angle'])
    df['wind_angle'] = np.where(df['wind'].str.contains("In From LF"), 135, df['wind_angle'])
    df['wind_angle'] = np.where(df['wind'].str.contains("In From CF"), 180, df['wind_angle'])
    df['wind_angle'] = np.where(df['wind'].str.contains("In From RF"), -135, df['wind_angle'])
    df['wind_angle'] = np.where(df['wind'].str.contains("R To L"), -90, df['wind_angle'])
    df['wind_angle'] = np.where(df['wind'].str.contains("Out To LF"), -45, df['wind_angle'])
    
    df['wind_angle'] = df['CF_angle'] + df['wind_angle'] 
    
    # Determine angle between them
    df['angle'] = df['wind_angle'] - df['CF_angle'] 

    # Calculate vectors
    df['x_vect'] = round(math.sin(math.radians(df['angle'])),5) * df['Speed']
    df['y_vect'] =round(math.cos(math.radians(df['angle'])),5) * df['Speed']

    all_parks = create_park_dummies()
    
    df = df.merge(all_parks, on='venue_id', how='inner')
    
    return df

In [10]:
# for filename in os.listdir(r"C:\Users\james\Documents\MLB\Data\FanGraphs\Batters"):
#     date = filename[11:19]
    
#     matchup_path = "Matchups" + date
    
#     df_list = []
#     for matchup in os.listdir(os.path.join(baseball_path, "New_Matchups", matchup_path)):
#         df = create_weather(matchup_path, matchup)
#         df_list.append(df)
    
#     all_df = pd.concat(df_list)
    

In [14]:
def daily_weather(date):
    
    matchup_path = "Matchups" + date
    
    df_list = []
    for matchup in os.listdir(os.path.join(baseball_path, "New_Matchups", matchup_path)):
        df = create_weather(matchup_path, matchup)
        df_list.append(df)
        
    all_df = pd.concat(df_list)
    
    all_df.drop_duplicates(subset=['venue_id'], keep='last', inplace=True)
    all_df = all_df.reset_index(drop=True)
   
    return all_df

all_df = daily_weather("20220513")
all_df

,venue_id,game_date,weather,wind,BBREFTEAM,CF,TEMP_PARK_CT,Speed,CF_angle,wind_angle,...,venue_3309,venue_3312,venue_3313,venue_4169,venue_4705,venue_5010,venue_5325,venue_5365,venue_5381,venue_5445
0,2394,2022-05-13,"82 degrees, Clear.","9 mph, R To L.",DET,SE,82,9,135.0,45.0,...,0,0,0,0,0,0,0,0,0,0
1,5325,2022-05-13,"72 degrees, Roof Closed.","0 mph, None.",TEX,SE,72,0,135.0,135.0,...,0,0,0,0,0,0,1,0,0,0
2,15,2022-05-13,"95 degrees, Clear.","8 mph, L To R.",ARI,N,95,8,0.0,90.0,...,0,0,0,0,0,0,0,0,0,0
3,3312,2022-05-13,"75 degrees, Partly Cloudy.","9 mph, Out To CF.",MIN,E,75,9,90.0,90.0,...,0,1,0,0,0,0,0,0,0,0
4,3309,2022-05-13,"74 degrees, Cloudy.","7 mph, R To L.",WSN,NE,74,7,45.0,-45.0,...,1,0,0,0,0,0,0,0,0,0
5,19,2022-05-13,"75 degrees, Cloudy.","6 mph, R To L.",COL,N,75,6,0.0,-90.0,...,0,0,0,0,0,0,0,0,0,0
6,10,2022-05-13,"65 degrees, Clear.","14 mph, Varies.",OAK,ENE,65,14,67.5,67.5,...,0,0,0,0,0,0,0,0,0,0
7,4,2022-05-13,"85 degrees, Clear.","9 mph, L To R.",CHW,SE,85,9,135.0,225.0,...,0,0,0,0,0,0,0,0,0,0
8,22,2022-05-13,"75 degrees, Clear.","5 mph, L To R.",LAD,NE,75,5,45.0,135.0,...,0,0,0,0,0,0,0,0,0,0
9,4705,2022-05-13,"77 degrees, Partly Cloudy.","8 mph, L To R.",SDP,N,77,8,0.0,90.0,...,0,0,0,0,1,0,0,0,0,0


In [21]:
date = "20220819"
file = "Daily_Weather_" + date + ".xlsx"

all_df = daily_weather(date)
all_df.to_excel(os.path.join(baseball_path, "Daily Weather2", file))

In [13]:
# Loop over dates that we have fangraphs projections for
for filename in os.listdir(r"C:\Users\james\Documents\MLB\Data\Rosters\Depth"):
    date = filename[5:13]
    file = "Daily_Weather_" + date + ".xlsx"
    
    print(date)
    try:
        all_df = daily_weather(date)
        all_df.to_excel(os.path.join(baseball_path, "Daily Weather2", file))
    except:
        print("Didn't work")

20220401
Didn't work
20220402
Didn't work
20220403
Didn't work
20220404
Didn't work
20220405
Didn't work
20220406
Didn't work
20220407
Didn't work
20220408
20220409
20220410
20220411
20220412
20220413
20220414
20220415
20220416
20220417
20220418
20220419
20220420
20220421
20220422
20220423
20220424
20220425
20220426
20220427
20220428
20220429
20220430
20220501
20220502
20220503
20220504
20220505
20220506
20220507
20220508
20220509
20220510
20220511
20220512
20220513
20220514
20220515
20220516
20220517
20220518
20220519
20220520
20220521
Didn't work
20220522
20220523
20220524
20220525
20220526
20220527
Didn't work
20220528
Didn't work
20220529
20220530
20220531
20220601
20220602
20220603
20220604
20220605
20220606
20220607
20220608
20220609
20220610
20220611
20220612
20220613
20220614
20220615
20220616
20220617
20220618
20220619
20220620
20220621
20220622
20220623
20220624
Didn't work
20220625
20220626
20220627
20220628
20220629
20220630
20220701
20220702
20220703
20220704
20220705
2022